<a href="https://colab.research.google.com/github/Snapchik/chat_analysis/blob/main/Chats_copmparisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming

In [ ]:
# download the stopwords from NLTK
nltk.download('stopwords')

nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hammas_chat = pd.read_json('/content/drive/My Drive/Colab Notebooks/result.json')

hammas_chat['hashtags'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['date'] = [[] for _ in range(len(hammas_chat))]
hammas_chat['clear_message'] = [[] for _ in range(len(hammas_chat))]

def extract_hashtags(messages):
    try:
        return re.findall(r'#(\w+)', str(messages))[0]
    except IndexError as err:
        return np.nan


hammas_chat['hashtags'] = hammas_chat['messages'].apply(extract_hashtags)

message_count = len(hammas_chat['messages'])

for index in range(message_count):
  hammas_chat['date'][index] = pd.to_datetime(hammas_chat.iloc[index, 3]['date'])

for index in range(message_count):
  hammas_chat['clear_message'][index] = hammas_chat.iloc[index, 3]['text']



hammas_chat = hammas_chat.drop(columns=['name', 'type', 'id'])
hammas_chat = hammas_chat.replace('', np.nan)
hammas_chat = hammas_chat.apply(lambda y: np.nan if len(y)==0 else y)
hammas_chat = hammas_chat.dropna(subset = ['clear_message'])

In [ ]:
# WHY THIS CODE WORKS AND NOT THE ONE ON TOP
for j in range(len(hammas_chat)):
  lst = []
  for i in hammas_chat.iloc[j, 3]:
    try:
      lst.extend(i.values())
    except AttributeError as err:
      lst.extend(i)
  lst = ''.join(lst)
  #getting rid of typetext/n
  typetext = '\w+\n'
  re.sub(typetext,'', lst)
  hammas_chat.iloc[j, 3] = lst

In [ ]:
#clearing out some common misspellings and hyperlink
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('\n', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('text_link', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'https?://[^\s\n\r]+', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'bold\B', '', regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace(r'#', '',regex=True)
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('hashtag', '')
hammas_chat['clear_message'] = hammas_chat['clear_message'].str.replace('italic', '')


In [ ]:
#applying tokenizer
hammas_chat['messages_token'] = hammas_chat["clear_message"].apply(nltk.word_tokenize)



*   Next I need to get rid of emojies
*   Than try to build a summarizer



In [ ]:
hammas_chat['clear_message'].str.findall('_')

2                                    []
4                                    []
5                                [_, _]
6                                    []
7        [_, _, _, _, _, _, _, _, _, _]
                      ...              
30366                                []
30367                                []
30368                                []
30369                                []
30370                                []
Name: clear_message, Length: 25566, dtype: object

In [ ]:
#expression finds - {'type': 'bold', 'text': '

r = '{+\W\w+\W+\w+\W+\w+\W+'